In this work, we are going to be considering predicting the risk of diabetes among Men and Women of diffrent ages. Diabetes.csv is a clean dataset of 253,680 survey responses to the CDC's BRFSS2015. The target variable Diabetes has 3 classes. 0 is for no diabetes or only during pregnancy, 1 is for prediabetes, and 2 is for diabetes. There is class imbalance in the dataset. This dataset has 21 feature variables.

The goal of this project is to examine Logistics regression model and it performance in classifiying whether an individual has diabetes or not.

In [1]:
#importing necessary libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import matplotlib.colors
import matplotlib.ticker as mtick

# feature selection
from sklearn.feature_selection import SelectKBest, mutual_info_classif, f_regression, f_classif, chi2

# feature scaler
from sklearn.preprocessing import StandardScaler, OrdinalEncoder

# for fixing the imbalanced dataset and split
from sklearn.model_selection import train_test_split
# from imblearn.under_sampling import RandomUnderSampler
# from imblearn.under_sampling import NearMiss
# from imblearn.over_sampling import SMOTE

# for model evaluation
from sklearn import metrics # Scores the performance of the model
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_validate

#model selection
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

# for the gridsearch
from sklearn.model_selection import GridSearchCV

In [2]:
# from google.colab import files
# # Using the files.upload() function to upload the dataset file
# uploaded = files.upload()

The Data

The dataset was sourced from kaggle and in three parts.

df1: dataset with data from patients without diabetes, pre-diabetes and diabetes.

df2: balanced patients with and without diabetes.

df3: imbalanced dataset of patients with and without diabetes.

This project will be mainly on the last 2, where we will explore df3 imbalanced and large dataset to understand the features and target relationship. We will use this information to predict patients in balanced dataset df2.

In [3]:
# from google.colab import files
# uploaded =files.upload()

In [4]:
# from google.colab import files
# uploaded =files.upload()

In [5]:
#reading the uploaded dataset into a dataframe
diabetes2 = pd.read_csv('./Data/diabetes2.csv')

In [6]:
#reading the uploaded dataset into a dataframe
diabetes3 = pd.read_csv('./Data/diabetes3.csv')

In [7]:
#To get the shape of the dataframe.
print('diabetes2 shape is: {}'.format(diabetes2.shape))
print('diabetes3 shape is: {}'.format(diabetes3.shape))

diabetes2 shape is: (70692, 22)
diabetes3 shape is: (253680, 22)


We are going to have a quick look at the dataset before cleaning it up.

In [8]:
diabetes3.head()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


In [9]:
diabetes3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253680 entries, 0 to 253679
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Diabetes_binary       253680 non-null  float64
 1   HighBP                253680 non-null  float64
 2   HighChol              253680 non-null  float64
 3   CholCheck             253680 non-null  float64
 4   BMI                   253680 non-null  float64
 5   Smoker                253680 non-null  float64
 6   Stroke                253680 non-null  float64
 7   HeartDiseaseorAttack  253680 non-null  float64
 8   PhysActivity          253680 non-null  float64
 9   Fruits                253680 non-null  float64
 10  Veggies               253680 non-null  float64
 11  HvyAlcoholConsump     253680 non-null  float64
 12  AnyHealthcare         253680 non-null  float64
 13  NoDocbcCost           253680 non-null  float64
 14  GenHlth               253680 non-null  float64
 15  

Now we are about to start the cleaning process of the dataset, and will we start by checking for missing values, before moving to duplicates values, and the rest.

In [10]:
diabetes3.isna().sum()

Diabetes_binary         0
HighBP                  0
HighChol                0
CholCheck               0
BMI                     0
Smoker                  0
Stroke                  0
HeartDiseaseorAttack    0
PhysActivity            0
Fruits                  0
Veggies                 0
HvyAlcoholConsump       0
AnyHealthcare           0
NoDocbcCost             0
GenHlth                 0
MentHlth                0
PhysHlth                0
DiffWalk                0
Sex                     0
Age                     0
Education               0
Income                  0
dtype: int64

No missing data in the dataset. so we head on to finding if their are any duplicate values.

In [11]:
diabetes3.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
Diabetes_binary,253680.0,0.139333,0.346294,0.0,0.0,0.0,0.0,1.0
HighBP,253680.0,0.429001,0.494934,0.0,0.0,0.0,1.0,1.0
HighChol,253680.0,0.424121,0.494210,0.0,0.0,0.0,1.0,1.0
CholCheck,253680.0,0.962670,0.189571,0.0,1.0,1.0,1.0,1.0
BMI,253680.0,28.382364,6.608694,12.0,24.0,27.0,31.0,98.0
Smoker,253680.0,0.443169,0.496761,0.0,0.0,0.0,1.0,1.0
Stroke,253680.0,0.040571,0.197294,0.0,0.0,0.0,0.0,1.0
HeartDiseaseorAttack,253680.0,0.094186,0.292087,0.0,0.0,0.0,0.0,1.0
PhysActivity,253680.0,0.756544,0.429169,0.0,1.0,1.0,1.0,1.0
Fruits,253680.0,0.634256,0.481639,0.0,0.0,1.0,1.0,1.0


In [12]:
#check for duplicates rows, and show the first 5 rows duplicated.
duplicates = diabetes3[diabetes3.duplicated()]
print("Duplicate Rows : ",len(duplicates))
duplicates.head()

Duplicate Rows :  24206


,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
1242,1.0,1.0,1.0,1.0,27.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,0.0,30.0,1.0,0.0,10.0,4.0,5.0
1563,0.0,0.0,0.0,1.0,21.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,6.0,8.0
2700,0.0,0.0,0.0,1.0,32.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,5.0,6.0,8.0
3160,0.0,0.0,0.0,1.0,21.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,6.0,8.0
3332,0.0,0.0,0.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,9.0,6.0,8.0


In [13]:
#Removing duplicates.

def data_clean(dataframe):

    bins = [0, 18.5, 24.9, 29.9, float('inf')]
    labels = [1,2,3,4]

    df = (dataframe
         .drop_duplicates()
         .assign(BMI_bins=dataframe.BMI.pipe(pd.cut, bins=bins, labels=labels))
         .reset_index(drop=True)
         .astype({'Diabetes_binary':'uint8',
     'HighBP':'uint8',
     'HighChol':'uint8',
     'CholCheck':'uint8',
     'BMI':'uint8',
     'Smoker':'uint8',
     'Stroke':'uint8',
     'HeartDiseaseorAttack':'uint8',
     'PhysActivity':'uint8',
     'Fruits':'uint8',
     'Veggies':'uint8',
     'HvyAlcoholConsump':'uint8',
     'AnyHealthcare':'uint8',
     'NoDocbcCost':'uint8',
     'GenHlth':'uint8',
     'MentHlth':'uint8',
     'PhysHlth':'uint8',
     'DiffWalk':'uint8',
     'Sex':'uint8',
     'Age':'uint8',
     'Education':'uint8',
     'Income':'uint8',
     'BMI_bins':'uint8'})


         )
    return df

Using uint8 reduces DataFrame memory size without losing information and helps for classification models interpretation later.

In [14]:
df = data_clean(diabetes3)
cleaned_df2 = data_clean(diabetes2)

In [ ]:
cleaned_df2.to_csv("./Data/cleaned_diabetes2.csv")

After dataset cleanup

In [15]:
cleaned_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69057 entries, 0 to 69056
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   Diabetes_binary       69057 non-null  uint8
 1   HighBP                69057 non-null  uint8
 2   HighChol              69057 non-null  uint8
 3   CholCheck             69057 non-null  uint8
 4   BMI                   69057 non-null  uint8
 5   Smoker                69057 non-null  uint8
 6   Stroke                69057 non-null  uint8
 7   HeartDiseaseorAttack  69057 non-null  uint8
 8   PhysActivity          69057 non-null  uint8
 9   Fruits                69057 non-null  uint8
 10  Veggies               69057 non-null  uint8
 11  HvyAlcoholConsump     69057 non-null  uint8
 12  AnyHealthcare         69057 non-null  uint8
 13  NoDocbcCost           69057 non-null  uint8
 14  GenHlth               69057 non-null  uint8
 15  MentHlth              69057 non-null  uint8
 16  Phys

In [16]:
cleaned_df2.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
Diabetes_binary,69057.0,0.508232,0.499936,0.0,0.0,1.0,1.0,1.0
HighBP,69057.0,0.571224,0.494905,0.0,0.0,1.0,1.0,1.0
HighChol,69057.0,0.531329,0.499021,0.0,0.0,1.0,1.0,1.0
CholCheck,69057.0,0.974803,0.156723,0.0,1.0,1.0,1.0,1.0
BMI,69057.0,29.955834,7.147972,12.0,25.0,29.0,33.0,98.0
Smoker,69057.0,0.481935,0.499677,0.0,0.0,0.0,1.0,1.0
Stroke,69057.0,0.063643,0.244118,0.0,0.0,0.0,0.0,1.0
HeartDiseaseorAttack,69057.0,0.150875,0.357930,0.0,0.0,0.0,0.0,1.0
PhysActivity,69057.0,0.696483,0.459780,0.0,0.0,1.0,1.0,1.0
Fruits,69057.0,0.605659,0.488712,0.0,0.0,1.0,1.0,1.0


## Modelling

In [17]:
cleaned_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69057 entries, 0 to 69056
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   Diabetes_binary       69057 non-null  uint8
 1   HighBP                69057 non-null  uint8
 2   HighChol              69057 non-null  uint8
 3   CholCheck             69057 non-null  uint8
 4   BMI                   69057 non-null  uint8
 5   Smoker                69057 non-null  uint8
 6   Stroke                69057 non-null  uint8
 7   HeartDiseaseorAttack  69057 non-null  uint8
 8   PhysActivity          69057 non-null  uint8
 9   Fruits                69057 non-null  uint8
 10  Veggies               69057 non-null  uint8
 11  HvyAlcoholConsump     69057 non-null  uint8
 12  AnyHealthcare         69057 non-null  uint8
 13  NoDocbcCost           69057 non-null  uint8
 14  GenHlth               69057 non-null  uint8
 15  MentHlth              69057 non-null  uint8
 16  Phys

In [25]:
cleaned_df2.Diabetes_binary.unique()

array([0, 1], dtype=uint8)

In [19]:
X = cleaned_df2.drop('Diabetes_binary', axis=1)
y = cleaned_df2['Diabetes_binary']

pop()
create y first 

In [20]:

X_train, X_test, y_train, y_test = train_test_split(  X, y, test_size=0.3, random_state=44)

In [21]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        pred = clf.predict(X_train)
        clf_report = pd.DataFrame(classification_report(y_train, pred, output_dict=True))
        tn, fp, fn, tp = confusion_matrix(y_train, pred).ravel()
        specificity = tn / (tn + fp)
        sensitivity = tp / (tp + fn)
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
        print(f"Specificity: {specificity * 100:.2f}%")
        print(f"Sensitivity: {sensitivity * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix:\n{confusion_matrix(y_train, pred)}\n")
        
    elif not train:
        pred = clf.predict(X_test)
        clf_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True))
        tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
        specificity = tn / (tn + fp)
        sensitivity = tp / (tp + fn)
        print("Test Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
        print(f"Specificity: {specificity * 100:.2f}%")
        print(f"Sensitivity: {sensitivity * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix:\n{confusion_matrix(y_test, pred)}\n")


In [22]:
# SVM
from sklearn.svm import SVC
print("=======================Linear Kernel SVM==========================")
model = SVC(kernel='linear')
model.fit(X_train, y_train)

print_score(model, X_train, y_train, X_test, y_test, train=True)
print_score(model, X_train, y_train, X_test, y_test, train=False)

print("=======================Polynomial Kernel SVM==========================")
from sklearn.svm import SVC

model = SVC(kernel='poly', degree=2, gamma='auto')
model.fit(X_train, y_train)

print_score(model, X_train, y_train, X_test, y_test, train=True)
print_score(model, X_train, y_train, X_test, y_test, train=False)

print("=======================Radial Kernel SVM==========================")
from sklearn.svm import SVC

model = SVC(kernel='rbf', gamma=1)
model.fit(X_train, y_train)

print_score(model, X_train, y_train, X_test, y_test, train=True)
print_score(model, X_train, y_train, X_test, y_test, train=False)

=======================Linear Kernel SVM==========================
Train Result:
Accuracy Score: 74.66%
Specificity: 69.72%
Sensitivity: 79.48%
_______________________________________________
CLASSIFICATION REPORT:
                      0             1  accuracy     macro avg  weighted avg
precision      0.768165      0.729055  0.746581      0.748610      0.748366
recall         0.697168      0.794777  0.746581      0.745973      0.746581
f1-score       0.730947      0.760499  0.746581      0.745723      0.745907
support    23868.000000  24471.000000  0.746581  48339.000000  48339.000000
_______________________________________________
Confusion Matrix:
[[16640  7228]
 [ 5022 19449]]

Test Result:
Accuracy Score: 74.35%
Specificity: 69.46%
Sensitivity: 78.99%
_______________________________________________
CLASSIFICATION REPORT:
                      0             1  accuracy     macro avg  weighted avg
precision      0.758412      0.731416   0.74346      0.744914      0.744566
recall  

In [24]:
#random forest hyperparameter tuning
from sklearn.ensemble import RandomForestClassifier
rf_param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

rf_grid = GridSearchCV(RandomForestClassifier(), rf_param_grid, cv=5)
rf_grid.fit(X_train, y_train)

clf = GridSearchCV(RandomForestClassifier(), 
                   param_grid = rf_param_grid, 
                   cv = 5)
clf.fit(X_train, y_train)


best_params = rf_grid.best_params_
print(f"Best params: {best_params}")

rf_clf =RandomForestClassifier(**best_params)
rf_clf.fit(X_train, y_train)
print_score(rf_clf, X_train, y_train, X_test, y_test, train=True)
print_score(rf_clf, X_train, y_train, X_test, y_test, train=False)

Best params: {'criterion': 'gini', 'max_depth': 8, 'max_features': 'log2', 'n_estimators': 200}
Train Result:
Accuracy Score: 75.55%
Specificity: 71.10%
Sensitivity: 79.90%
_______________________________________________
CLASSIFICATION REPORT:
                      0             1  accuracy     macro avg  weighted avg
precision      0.775265      0.739178  0.755518      0.757222      0.756996
recall         0.710952      0.798987  0.755518      0.754969      0.755518
f1-score       0.741717      0.767920  0.755518      0.754818      0.754982
support    23868.000000  24471.000000  0.755518  48339.000000  48339.000000
_______________________________________________
Confusion Matrix:
[[16969  6899]
 [ 4919 19552]]

Test Result:
Accuracy Score: 74.38%
Specificity: 70.19%
Sensitivity: 78.36%
_______________________________________________
CLASSIFICATION REPORT:
                      0             1  accuracy     macro avg  weighted avg
precision      0.754982      0.734627  0.743846      0.